In [1]:
import numpy as np
import pandas as pd

In [5]:
np.random.seed(1)
df = pd.DataFrame(np.random.randint(1, 100, (200000, 6)))
df.columns = ["语文", "数学", "英语", "物理", "化学", "生物"]
df.index = ["A"+str(i) for i in range(df.shape[0])]
df

,语文,数学,英语,物理,化学,生物
A0,38,13,73,10,76,6
A1,80,65,17,2,77,72
A2,7,26,51,21,19,85
A3,12,29,30,15,51,69
A4,88,88,95,97,87,14
...,...,...,...,...,...,...
A199995,42,93,96,24,41,95
A199996,5,15,54,42,44,44
A199997,1,3,42,47,94,31
A199998,78,97,38,70,17,14


现在需求是求每一行第二高的成绩和第二低的成绩

# 常规思维apply

In [6]:
def func(ser):
    ser = ser.sort_values()
    return pd.Series([ser[1], ser[-2]])

In [7]:
%%time
r = df.apply(func, axis=1)

Wall time: 3min 10s


In [8]:
r

,0,1
A0,10,73
A1,17,77
A2,19,51
A3,15,51
A4,87,95
...,...,...
A199995,41,95
A199996,15,44
A199997,3,47
A199998,17,78


# np元素级矢量化操作

- 1 np.vectorize(pyfunc), 返回一个能对ndarray进行元素级别矢量化操作的函数。
- 2 把每一行数据拼接在一起，放在列表里面，再把列表整体编程字符串。

In [9]:
df

,语文,数学,英语,物理,化学,生物
A0,38,13,73,10,76,6
A1,80,65,17,2,77,72
A2,7,26,51,21,19,85
A3,12,29,30,15,51,69
A4,88,88,95,97,87,14
...,...,...,...,...,...,...
A199995,42,93,96,24,41,95
A199996,5,15,54,42,44,44
A199997,1,3,42,47,94,31
A199998,78,97,38,70,17,14


In [10]:
arr = (df.astype(str)+",").sum(axis=1).str.split(',').str[0:-1].astype(str).values
arr

array(["['38', '13', '73', '10', '76', '6']",
       "['80', '65', '17', '2', '77', '72']",
       "['7', '26', '51', '21', '19', '85']", ...,
       "['1', '3', '42', '47', '94', '31']",
       "['78', '97', '38', '70', '17', '14']",
       "['51', '60', '2', '59', '30', '12']"], dtype=object)

In [11]:
x = arr[0]

In [12]:
x

"['38', '13', '73', '10', '76', '6']"

In [13]:
eval(x)

['38', '13', '73', '10', '76', '6']

In [14]:
np.array(eval(x))

array(['38', '13', '73', '10', '76', '6'], dtype='<U2')

In [15]:
np.array(eval(x)).astype(float)

array([38., 13., 73., 10., 76.,  6.])

In [16]:
np.array(eval(x)).astype(float)

array([38., 13., 73., 10., 76.,  6.])

In [17]:
# 定义处理单个x的函数
def pyfunc(x):
    # 把字符串变成列表，在转成ndarray把数据变成浮点数
    x = np.array(eval(x)).astype(float)
    # 排序,升序
    x = np.sort(x)
    return str([x[1], x[-2]])

In [18]:
pyfunc(x)

'[10.0, 73.0]'

In [19]:
# numpy矢量化函数
np_func = np.vectorize(pyfunc)

In [20]:
%%time
r2 = np_func(arr)
r2 = pd.Series(r2).str.strip("[]").str.split(',', expand=True)
r2.index = df.index
r2 = r2.astype(float)
r2

Wall time: 18.5 s


,0,1
A0,10.0,73.0
A1,17.0,77.0
A2,19.0,51.0
A3,15.0,51.0
A4,87.0,95.0
...,...,...
A199995,41.0,95.0
A199996,15.0,44.0
A199997,3.0,47.0
A199998,17.0,78.0


In [56]:
from numba import guvectorize

In [65]:
@guvectorize
def pyfunc2(x):
    # 把字符串变成列表，在转成ndarray把数据变成浮点数
    x = np.array(eval(x)).astype(float)
    # 排序,升序
    x = np.sort(x)
    return str([x[1], x[-2]])

In [66]:
%%time
r2 = pyfunc2(arr)

TypeError: The decorated object is not a function (got type <class 'numpy.ndarray'>).

In [31]:
# r2 = pd.Series(r2).str.strip("[]").str.split(',', expand=True)
# r2.index = df.index
# r2 = r2.astype(float)
# r2

In [32]:
vectorize?